<a href="https://colab.research.google.com/github/Rizukaf-id/challenge-ai-engineer-dot/blob/main/Challenge_AI_Engineer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge AI Engineer
oleh Rizka Fiddiyansyah

## Bagian 1: Praktik Pemrograman (Python dan Google Colab)

### Soal 1.b: Aplikasi Chatbot FAQ melalui CLI/Terminal

#### import library

In [3]:
import re
import torch
from transformers import AutoTokenizer, AutoModel

#### Load faq.txt

In [14]:
!wget https://raw.githubusercontent.com/Rizukaf-id/challenge-ai-engineer-dot/refs/heads/main/faq.txt -O faq.txt

--2025-05-21 13:25:29--  https://raw.githubusercontent.com/Rizukaf-id/challenge-ai-engineer-dot/refs/heads/main/faq.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2955 (2.9K) [text/plain]
Saving to: ‘faq.txt’

faq.txt             100%[===================>]   2.89K  --.-KB/s    in 0s      

2025-05-21 13:25:30 (35.3 MB/s) - ‘faq.txt’ saved [2955/2955]



In [33]:
with open('faq.txt', 'r', encoding='utf-8') as f:
    faq_raw = f.read().strip()

faq = re.findall(
    r'Q\s*[:：]\s*(.+?)\s*A\s*[:：]\s*(.+?)(?=(?:\nQ[:：]|$))',
    faq_raw, flags=re.DOTALL | re.IGNORECASE
)
faq = [(q.strip().replace('\n', ' '), a.strip().replace('\n', ' ')) for q, a in faq if q and a]

if not faq:
    print('FAQ kosong! Pastikan format FAQ.txt: Q: ... A: ...')
    exit(1)

#### Load Model

In [34]:
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

#### Alasan Pemilihan Model `sentence-transformers/all-MiniLM-L6-v2` untuk Chatbot FAQ

Dalam pembuatan chatbot FAQ ini, model **[sentence-transformers/all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2)** dipilih daripada model-model besar seperti Llama, Gemma, Phi, atau OpenChat. Hal ini didasarkan pada beberapa pertimbangan utama, terutama terkait efisiensi, kecepatan, dan keterbatasan sumber daya pada layanan gratis Google Colab.

Model MiniLM-L6-v2 memiliki ukuran yang jauh lebih kecil dan ringan dibandingkan large language model (LLM) generatif di atas, sehingga dapat berjalan dengan lancar di lingkungan dengan memori dan kapasitas komputasi terbatas tanpa memerlukan GPU besar. Selain itu, MiniLM-L6-v2 memang dirancang dan dioptimalkan khusus untuk menghasilkan embedding kalimat yang akurat, sangat ideal untuk kasus penggunaan chatbot berbasis pencocokan makna (semantic search) seperti ini. Dengan menggunakan MiniLM-L6-v2, chatbot dapat memberikan hasil pencarian jawaban yang relevan secara semantik dengan waktu respons yang cepat dan konsumsi resource yang minimal, sehingga sangat sesuai untuk implementasi di Google Colab gratis yang sering kali memiliki pembatasan RAM, CPU, dan runtime.

#### Build App

In [35]:
def get_embedding(text):
  encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

  with torch.no_grad():
    model_output = model(**encoded_input)

  token_embeddings = model_output.last_hidden_state
  attention_mask = encoded_input['attention_mask']
  input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
  sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
  sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
  return (sum_embeddings / sum_mask).squeeze(0)


# precompute FAQ embeddings
faq_questions = [q for q, a in faq]
faq_embeddings = torch.stack([get_embedding(q) for q in faq_questions])

def ans_question(query, cossim_threshold=0.72):
  query_embedding = get_embedding(query)

  # hitung cosine similarity
  similarities = torch.nn.functional.cosine_similarity(query_embedding.unsqueeze(0), faq_embeddings)
  top_score, top_idx = torch.max(similarities, dim=0)
  top_score = top_score.item()
  top_idx = top_idx.item()

  # print(f"[DEBUG] Similarity: {top_score:.3f}")

  if top_score >= cossim_threshold:
    return faq[top_idx][1]
  else:
    return 'Maaf, saya tidak dapat membantu dengan pertanyaan itu.'

def chat():
  print("📚 Chatbot Kursus Coding Anak")
  print("Ketik pertanyaan anda, atau ketik 'exit' untuk keluar.\n")

  while True:
    query = input('👤 : ')
    if query.lower() in ['exit', 'quit', 'keluar']:
      break
    answer = ans_question(query)
    print('🤖 : ', answer)

**Fungsi get_embedding**
   - Mengubah teks menjadi embedding vektor menggunakan model.
   - Embedding ini digunakan untuk mengukur kemiripan antar kalimat.

**Precompute FAQ Embeddings**
   - Semua pertanyaan FAQ diubah menjadi embedding dan disimpan dalam tensor.
   - Mempercepat pencarian jawaban karena tidak perlu menghitung ulang embedding setiap kali ada pertanyaan.

**Fungsi ans_question**
   - Menerima pertanyaan user, mengubahnya menjadi embedding, lalu menghitung cosine similarity dengan semua embedding pertanyaan FAQ.
   - Jawaban dengan kemiripan tertinggi diambil jika melewati threshold tertentu (0.72), jika tidak chatbot akan menjawab "Maaf, saya tidak dapat membantu dengan pertanyaan itu.".

#### Run APP

In [36]:
chat()

📚 Chatbot Kursus Coding Anak
Ketik pertanyaan anda, atau ketik 'exit' untuk keluar.

👤 : apa itu kursus koding?
[DEBUG] Similarity: 0.779
🤖 :  Layanan kursus koding untuk anak-anak adalah program edukasi yang mengenalkan dunia pemrograman secara menyenangkan dan interaktif untuk anak usia 7 hingga 15 tahun. Kami menggunakan pendekatan bermain sambil belajar agar anak-anak mudah memahami konsep teknologi.
👤 : exit


## Bagian 2: Teori AI

### Soal 2: Pertanyaan Teori Dasar

#### a. Apa yang dimaksud dengan Artificial Intelligence (AI)? Sebutkan dua contohnya dalam kehidupan sehari-hari.

Bla bla

#### b. Apa perbedaan antara Supervised Learning dan Unsupervised Learning? Berikan satu contoh untuk masing-masing.

bla bla

### Soal 3: Pertanyaan Konsep

#### a. Apa itu Feature dalam konteks machine learning? Mengapa penting untuk memilih fitur yang tepat saat membangun model?

blaafjawef

#### b. Apa itu Fine-tuning dalam machine learning? Sebutkan satu kasus di mana fine-tuning berguna.

asdf;ahdfh